# Modelo Preditivo de Fraude - Random Forest (MLLIB) FINAL V-7.8

## Objetivo
Criar modelo preditivo usando como features os eventos associados à fraude reportada na planilha de POC.

## Observações
Cada observação corresponde à contagem diária de eventos ocorridos ocorridos por agência, atm e tipo de evento.

## Features
Neste modelo, serão consideradas as seguintes features:
+ N features, para eventos "3321", "1689", "1617","5171", "3672", "1568","1615", "883", "2143", selecionados como mais relevantes
+ valor: índice de afastamento do número de ocorrências do evento na observação diária em relação à média diária do evento para cada par agência/atm (nro eventos X / média diária do evento X).

**OBS: o melhor resultado foi obtido com a melhor feature selecionada (3321)**

## Target
Identifica a ocorrência ou não de fraude para uma dada agência em uma determinada data, baseado em report efetuado na data de identificação do evento na planilha BASE PCO ATM.

## Libraries

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

#from pyspark import SparkContext
#sc = SparkContext()



## Definição de constantes

In [2]:
# Load extrato from HDFS into Dataframe
poc_in = '/user/x181472/export/V10_BASE_POC_ATM_060916.csv'
poceven_in = '/temporario/pokeventos-V2.csv'
ranking_poc_atm = '/temporario/rankingPocATM.csv'
marcador_poc_atm = '/user/x181472/marcadorPocATM.csv'

## Definição de funções

In [3]:
####################################################################################
# PARA RODAR EM HADOOP PRODUCAO DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO MAC OSX 
####################################################################################
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

## Trata Carga de Eventos de ATM

In [4]:
rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')

df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')

root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


## Prepara Datas Usadas no Processamento

In [5]:
import datetime, time
from dateutil.relativedelta import relativedelta
data_ref = datetime.datetime(2016, 8, 22)
mediana = 50  # mediana da ocorrência de eventos diários (feature)
percentil_max = 95 # percentil máximo considerado para treinar o modelo
D_90 = 90 # 90 dias a subtrair (usado somente para treino inicial)
D_1 = 1 # obtem eventos de D-1 para previsão de POC em produção
D_2 = 2 # obtem eventos a aprtir de D-2 para trás para treino constante do modelo em produção

# obtendo faixa do período de histórico a partir da data de referencia 
D_menos1 = data_ref + relativedelta(days=-D_1)
D_menos2 = data_ref + relativedelta(days=-D_2)
D_menos90 = D_menos2 + relativedelta(days=-D_90)
print "D-90:   " + str(D_menos90)
print "D-1 :   " + str(D_menos1)
print "D-2 :   " + str(D_menos2)

dt_sql_90 = (time.mktime(datetime.datetime.strptime(str(D_menos90.date()), "%Y-%m-%d").timetuple()))
dt_sql_1 = (time.mktime(datetime.datetime.strptime(str(D_menos1.date()), "%Y-%m-%d").timetuple()))
dt_sql_2 = (time.mktime(datetime.datetime.strptime(str(D_menos2.date()), "%Y-%m-%d").timetuple()))
dt_sql_ref = (time.mktime(datetime.datetime.strptime(str(data_ref.date()), "%Y-%m-%d").timetuple()))
print dt_sql_90
print dt_sql_1
print dt_sql_2
print dt_sql_ref

D-90:   2016-05-22 00:00:00
D-1 :   2016-08-21 00:00:00
D-2 :   2016-08-20 00:00:00
1463886000.0
1471748400.0
1471662000.0
1471834800.0


## Carga de dados brutos
Os dados brutos de log das ATMs e o de ocorrência de POCs carregados e descritos abaixo

### Dados de log de eventos

In [6]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

#df_eventlog.printSchema()   

# Elimina NAs
df_eventlog = df_eventlog.dropna()

# FILTRANDO SOMENTE ULTIMOS 90 DIAS, incluindo log de D-2
df_eventlog2 = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                   .format(dt_sql_90, dt_sql_1)).orderBy("dh_even")

## Obtendo Eventos Marcadores de POC (selecionados como features)
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.
São considerados fora do padrão, eventos que ocorrem em dias de ocorrência de POC, em agências reportadas como tendo ocorrência de POC, e que têm a quantidade de ocorrências no dia acima do percentil 95% da distribuição diária do eventos em todas as ATMs. Além disso, devem ser eventos que não aparecem com valores maiores que o percentil 95% nos dias em que não existem ocorrências de POCs.

In [7]:
# calculando percentis de baseline para todas as observações do log com base em agencia/evento
# OBS: o uso do percentil desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem

#df_eventlog_percen = df_eventlog.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
#                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
#                                .select(['id', 'nr_eventos'])

df_eventlog_percen = df_eventlog2.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [8]:
#df_eventlog_percen.limit(10).orderBy(col('nr_eventos').desc()).toPandas()

In [9]:
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

In [10]:
rdd_eventlog_percen.first()

(u'1053-2032', 1.0, 2.0)

In [11]:
rdd_eventlog_percen = rdd_eventlog_percen \
   .map(lambda k: (k[0].split("-"), k[1], k[2])) 

#rdd_eventlog_percen = rdd_eventlog_percen \
#   .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [12]:
rdd_eventlog_percen.first()

([u'1053', u'2032'], 1.0, 2.0)

In [13]:
#df_eventlog_percen = rdd_eventlog_percen \
#    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
#    .toDF(['nr_idef_secu',
#           'cd_eqpt',
#           'nr_sequ_even',
#           'mediana',
#           'percentil_max'
#                   ])
# dataframe por agencia/evento com valor de percentil_max
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1], float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'nr_sequ_even',
           'percentil_max'
                   ])

In [14]:
df_eventlog_percen.limit(10).toPandas()

nr_idef_secu nr_sequ_even  percentil_max
0         1217         6765           1.10
1         1561         3367           1.00
2         4268         3443           1.00
3         3412         3110           1.00
4         3055         6944           1.55
5         1525         6793           2.00
6          245         6798           1.00
7         3063         3009           5.00
8          458         2430           2.00
9         1610         1627           1.00

In [15]:
df_eventlog2.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [16]:
# colocar no log total as colunas de mediana e percentil máximo por agencia+evento 
#df_eventlog2 = df_eventlog2 \
#    .join(df_eventlog_percen, (df_eventlog['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
#                              (df_eventlog['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
#                              (df_eventlog['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
#    .drop(df_eventlog_percen["nr_idef_secu"])\
#    .drop(df_eventlog_percen["cd_eqpt"])\
#    .drop(df_eventlog_percen["nr_sequ_even"])

df_eventlog2 = df_eventlog2 \
    .join(df_eventlog_percen, (df_eventlog2['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_eventlog2['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [18]:
df_eventlog2 = df_eventlog2.dropna()

In [19]:
df_eventlog2.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  percentil_max
0          105     158         1583 2016-07-04           1            1.0
1          105     171          933 2016-06-07           4            3.7
2          105     166          933 2016-06-07           4            3.7
3          105     151          933 2016-07-30           1            3.7
4          105     167          933 2016-06-03           1            3.7
5          105     170          933 2016-06-03           1            3.7
6          105     171          933 2016-06-03           2            3.7
7          105     172          933 2016-06-03           1            3.7
8          105     173          933 2016-06-03           1            3.7
9          105     164          933 2016-06-03           1            3.7

In [20]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
#df_eventlog2 = df_eventlog2\
#    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
#    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda

df_eventlog2 = df_eventlog2\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda

### Dados de registro de POC
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

In [21]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
# pegando data inicial de comprometimento ao inves de data_cad, pois analise exploratoria mostrou que tem
# mais sentido usar esta data
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(";")) \
    .map(lambda p: (p[0], p[3], p[7])) \
    .toDF(['data_range', 
            'agencia',
            'golpe'
        ])
    
#df_poc.limit(10).toPandas()

df_poc = df_poc.withColumn('data_range', funcStrToDate_dmyyyy(col('data_range'))) 
df_poc.printSchema()
#df_poc.limit(10).toPandas()

df_poc = df_poc \
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_range', df_poc["data_range"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType()))
#df_poc.printSchema()

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("CAST(data_range AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_2)).orderBy("data_range")
#df_eventlog_TRN = df_eventlog_TRN.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
#df_poc.limit(10).toPandas()

# checking NAs
print ("\nNúmero de linhas carregadas: " + str("%d" % (df_poc.count())))
df_poc = df_poc.dropna()
print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_poc.count())))



Número de linhas carregadas: 903
root
 |-- data_range: date (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)


Número de linhas carregadas: 448

Número de linhas após retirar NAs: 448


## Selecionando Eventos por ATM/Agência em dias POC

In [22]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
df_poceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_range'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')

# LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
df_pocevenSIM = df_poceven\
    .filter((col("nr_eventos") > col('percentil_max')) &
            ((col("agencia") == col('nr_idef_secu'))))

## Selecionando Eventos por ATM/Agência Fora de dias POC

In [ ]:
df_pocevenSIM.limit(10).toPandas()

In [23]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
# join left garante pegar eventos que ocorrem em dia de POC e não depois
df_pocevenNAO = df_poceven\
    .filter((col("nr_eventos") <= col('percentil_max')) &
            (col('agencia').isNull())) # indica que foi fora de dia de POC

# JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
df_pokeventos = df_pocevenSIM\
    .join(df_pocevenNAO,
          (df_pocevenSIM['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'left')\
    .drop(df_pocevenNAO["nr_sequ_even"])\
    .groupBy('nr_sequ_even').count()

In [14]:
df_pocevenNAO.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  mediana  \
0         3544     152         3408 2016-05-25           2      1.0   
1         3544     169         6364 2016-05-25           1      1.5   
2         3544     170         3011 2016-05-25           1      1.0   
3         3544     158         1621 2016-05-25           2      1.0   
4         3544     162          724 2016-05-25           1      2.0   
5         3544     164         3404 2016-05-25           2      2.0   
6         3544     170         3405 2016-05-25           8      2.0   
7         3544     152         2412 2016-05-25           3      2.0   
8         3544     153          929 2016-05-25           1      1.0   
9         3544     154         4236 2016-05-25          10      2.0   

   percentil_max data_range agencia golpe  
0           4.65       None    None  None  
1           1.95       None    None  None  
2           2.00       None    None  None  
3           4.00       None    None  None  
4           3.00       None    None  None  
5           9.40       None    None  None  
6          14.90       None    None  None  
7           6.05       None    None  None  
8           1.00       None    None  None  
9          12.10       None    None  None

### Obtem lista de pokeventos

In [76]:
# gera lista de eventos para serem usados com features
#l_evenpoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
#len(l_evenpoc)
# 1846 SOMENTE PARA FORCAR TESTE d-2. tirar depois
#l_features1 = l_evenpoc[0:50]
#l_features2 = l_evenpoc[101:]
l_features1 = ["5840", "5251", "1649", "1746", "5242", "6740", "2184", "5241", "5248", "5198", "1846"] 
len(l_features1)

#len(l_features2)

11

### Dados de Ocorrências de POC

In [77]:
# USO 1
# FILTRANDO SOMENTE D-1 # DEPOIS COLOCAR ESTE TRY/CATCH ENCAPSULAR TODO O CODIGO FINAL
# TAMBEM SELECIONAR AQUI SOMENTE AGENCIAS QUE TEM OCORRENCIA DOS POKEVENTOS (AO MENOS 1)
try:
    df_eventlog_USO = df_eventlog.where("CAST(dh_even AS INT) = {0}"\
                                   .format(dt_sql_2)).orderBy("dh_even")
    df_eventlog_USO = df_eventlog_USO.filter((df_eventlog_USO["nr_sequ_even"].isin(l_features1)))
    
    if df_eventlog_USO.count() == 0:
        raise Exception("Nao existe nenhum evento relevante em D-1", "INFO-01")
        
except Exception as error:
        print('informação: ' + repr(error))

In [78]:
df_eventlog_USO.toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos
0         4664   10001         1846 2016-08-20           1
1         4433   10001         1846 2016-08-20           1
2          188   10001         1846 2016-08-20           2

### Obtém Marcadores de POC

In [80]:
# ver se tem sentido query abaixo
# aqui obtém somente eventos marcadores de POC para agencias que tiveram POC
# LISTA DE POKEVENTOS AGRUPADO POR Agencia+atm+evento

df_poctmp = df_pocevenSIM\
    .join(df_pocevenNAO,
          (df_pocevenSIM['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'inner')\
    .drop(df_pocevenNAO["nr_idef_secu"])\
    .drop(df_pocevenNAO["cd_eqpt"])\
    .drop(df_pocevenNAO["dh_even"])\
    .drop(df_pocevenNAO["nr_eventos"])\
    .drop(df_pocevenNAO["data_range"])\
    .drop(df_pocevenNAO["agencia"])\
    .drop(df_pocevenNAO["nr_sequ_even"])
# aqui obtem o percentilmax de cada agencia/evento

# ou fazer join por evento, mas selecionar agencia, evento, nr_evento e percentil_max
# selecionados os eventos + percentil_max (do evento somente?)

In [81]:
df_poctmp.limit(10).toPandas() # porque percentil_max dif

nr_sequ_even  percentil_max nr_idef_secu cd_eqpt nr_sequ_even    dh_even  \
0         1642            2.0          803     153         1642 2016-05-24   
1         1642            2.0         2226     151         1642 2016-05-30   
2         1642            2.0         1525     153         1642 2016-06-07   
3         1642            2.0          652     159         1642 2016-06-28   
4         1642            2.0         4433     154         1642 2016-07-03   
5         1642            2.0          206     153         1642 2016-07-21   
6         1642            2.0         1227     155         1642 2016-07-25   
7         1642            2.0         1227     166         1642 2016-07-25   
8         1642            2.0          436     154         1642 2016-07-28   
9         1642            2.0          436     155         1642 2016-07-28   

   nr_eventos  percentil_max data_range agencia golpe  
0           1           2.40       None    None  None  
1           1           3.00       None    None  None  
2           1          13.00       None    None  None  
3           4           8.10       None    None  None  
4           1           3.10       None    None  None  
5           1           2.95       None    None  None  
6           1           5.00       None    None  None  
7           1           5.00       None    None  None  
8           1           2.00       None    None  None  
9           2           2.00       None    None  None

In [30]:
df_x = df_x.groupBy(['nr_idef_secu','nr_sequ_even', 'percentil_max']).count()
    
df_x.limit(10).toPandas()

# falta: ler o D-1 geral (sem filtro de pokeventos) e filtrar para agencia+evento com nr-eventos > percentil_max

Empty DataFrame
Columns: [nr_idef_secu, nr_sequ_even, percentil_max, count]
Index: []

In [31]:
df_x.count()

0

In [16]:
#####################################################################################
# GERAR AQUI DATAFRAME PARA SAIDA DOS DADOS (agrupado por agencia/atm/nr_eventos)
#####################################################################################

In [17]:
#df_eventlog.count()

In [18]:
#df_eventlog2.count()

### Seleção dos registros de log para treino do modelo a partir dos eventos candidatos a POC

In [19]:
# com as features abaixo e cross validation. AUC:.76 (2 folders)
#l_features = ["3321", "1689", "1617","5171", "3672", "1568","1615", "3321", "2143"]
#l_features = ["1599", "1642", "1648","1650", "1651", "1653","1656", "1657", "1658",
#             "1765", "1769", "1813","2034", "2267", "2422","2430", "2492", "2493"]




# TESTAR TROCAR ABAIXO POR df_evenlog2
df_eventlog_TRN1 = df_eventlog2.filter((df_eventlog2["nr_sequ_even"].isin(l_features1)))
df_eventlog_TRN1.count()

3686

In [20]:
# Gerar uma transposicao no Pandas para cada lote abaixo e concatenar no dataframe spark depois para ML
# 992 evento quebrados em lotes de 150 PARA IDENTIFICAR LOTE COM FEATURE RELEVANTE
# necessario quebrar o numero de features abaixo por problema de memoria com Pandas
#l_evenpoc = df_eventlog2.groupBy(['nr_sequ_even']).count().toPandas().nr_sequ_even.tolist()
#l_features1 = l_evenpoc[0:150] # AUC: .64
#l_features2 = l_evenpoc[151:300] # AUC: .63
#l_features3 = l_evenpoc[301:450] # AUC: .66
#l_features4 = l_evenpoc[451:600] # AUC: .65
#l_features5 = l_evenpoc[601:750] # AUC: .62
#l_features6 = l_evenpoc[751:900] # AUC: .66
#l_features7 = l_evenpoc[901:] # AUC: .68

# obs. com cross fold a memoria nao é suficiente

## Preparando dados processados
Preparação do dataframe com as features e o target para treino do modelo


In [21]:
# obter as colunas de med diária por ag/equip/evento a partir do log todo
# media
df_even_med = df_eventlog2\
    .groupBy(['nr_idef_secu','cd_eqpt','nr_sequ_even'])\
    .avg('nr_eventos').withColumnRenamed('avg(nr_eventos)', 'med')
#df_even_med.limit(10).toPandas()

# troquei abaixo de left para inner para tirar NaN (15-09-2016)
# count left: 166923   count inner: 97236
# tentar com dropNA para ver se igual a inner: 
# concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
df_feat_TRN1 = df_eventlog_TRN1 \
    .join(df_even_med, (df_eventlog_TRN1['nr_idef_secu'] == df_even_med['nr_idef_secu']) &
                      (df_eventlog_TRN1['cd_eqpt'] == df_even_med['cd_eqpt']) &
                      (df_eventlog_TRN1['nr_sequ_even'] == df_even_med['nr_sequ_even']), 'left') \
    .drop(df_even_med["nr_idef_secu"]).drop(df_even_med["cd_eqpt"]).drop(df_even_med["nr_sequ_even"])

#df_feat_TRN.limit(10).toPandas()

# criar coluna de indice de proximidade da media (iproxmed)
# calculado como número de ocorrências real diaria/média diaria, por agencia e atm
df_feat_TRN1 = df_feat_TRN1\
    .withColumn('iproxmed', col("nr_eventos")/col("med"))

# transpondo tipo de evento para gerar features
df_feat_TRN1 = df_feat_TRN1\
    .drop(df_feat_TRN1["med"]).drop(df_feat_TRN1["nr_eventos"])

In [22]:
df_feat_TRN1 = df_feat_TRN1.dropna()

In [23]:
pd_tmp_TRN1 = df_feat_TRN1.toPandas()

In [24]:
pd_feat_TRN1 = pd.pivot_table(pd_tmp_TRN1,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

pd_feat_TRN1.columns = [c[1] if c[1] else c[0] for c in pd_feat_TRN1.columns.tolist()]
pd_feat_TRN1 = pd_feat_TRN1.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
#pd_feat_TRN

In [25]:
# lista de features para treinar o modelo
# a partir das features restantes apos a transposicao
l_feat_final = pd_feat_TRN1.columns.tolist()
#l_feat_final

In [26]:
df_feat_alt_TRN1 = sqlCtx.createDataFrame(pd_feat_TRN1)

In [27]:
# reordenando as colunas
df_feat_alt_TRN1 = df_feat_alt_TRN1.select('nr_idef_secu', 'cd_eqpt','data_even', 
                                           *[col for col in df_feat_alt_TRN1.columns if (col != 'nr_idef_secu') &
                                     (col != 'cd_eqpt') &
                                     (col != 'data_even')])

In [28]:
df_feat_alt_TRN1.first()

Row(nr_idef_secu=u'3246', cd_eqpt=u'10001', data_even=u'2016-05-26', 1649=0, 1746=0.0, 1846=1.7142857142857142, 2184=0.0, 5198=0.0, 5241=0.0, 5242=0.0, 5248=0.0, 5251=0, 5840=0.0, 6740=0)

In [29]:
# removendo o dataframe Pandas da memoria
lst = [pd_feat_TRN1]
del pd_feat_TRN1 # dfs still in list
del lst     # memory release now

In [30]:
df_feat_alt_TRN1 = df_feat_alt_TRN1.withColumn('data_even', df_feat_alt_TRN1["data_even"].cast(TimestampType()))

In [32]:
# AQUI REPETIR PARA TRN2 E CONCATENAR ANTES DE APLICAR EM ML

In [33]:
df_eventlog_TRN2 = df_eventlog2.filter((df_eventlog["nr_sequ_even"].isin(l_features2)))

# obter as colunas de med diária por ag/equip/evento a partir do log todo
# media
#df_even_med = df_eventlog\
#    .groupBy(['nr_idef_secu','cd_eqpt','nr_sequ_even'])\
#    .avg('nr_eventos').withColumnRenamed('avg(nr_eventos)', 'med')
#df_even_med.limit(10).toPandas()

In [34]:
# troquei abaixo de left para inner para tirar NaN (15-09-2016)
# count left: 166923   count inner: 97236
# tentar com dropNA para ver se igual a inner: 
# concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
df_feat_TRN2 = df_eventlog_TRN2 \
    .join(df_even_med, (df_eventlog_TRN2['nr_idef_secu'] == df_even_med['nr_idef_secu']) &
                      (df_eventlog_TRN2['cd_eqpt'] == df_even_med['cd_eqpt']) &
                      (df_eventlog_TRN2['nr_sequ_even'] == df_even_med['nr_sequ_even']), 'left') \
    .drop(df_even_med["nr_idef_secu"]).drop(df_even_med["cd_eqpt"]).drop(df_even_med["nr_sequ_even"])

In [35]:
#df_feat_TRN.limit(10).toPandas()

# criar coluna de indice de proximidade da media (iproxmed)
# calculado como número de ocorrências real diaria/média diaria, por agencia e atm
df_feat_TRN2 = df_feat_TRN2\
    .withColumn('iproxmed', df_feat_TRN2["nr_eventos"]/df_feat_TRN2["med"])

In [36]:
# transpondo tipo de evento para gerar features
df_feat_TRN2 = df_feat_TRN2\
    .drop(df_feat_TRN2["med"]).drop(df_feat_TRN2["nr_eventos"])

In [37]:
df_feat_TRN2 = df_feat_TRN2.dropna()

pd_tmp_TRN2 = df_feat_TRN2.toPandas()

In [39]:
pd_feat_TRN2 = pd.pivot_table(pd_tmp_TRN2,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

pd_feat_TRN2.columns = [c[1] if c[1] else c[0] for c in pd_feat_TRN2.columns.tolist()]

In [41]:
pd_feat_TRN2 = pd_feat_TRN2.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)

In [43]:
l_feat_final2 = pd_feat_TRN2.columns.tolist()

152

In [44]:
df_feat_alt_TRN2 = sqlCtx.createDataFrame(pd_feat_TRN2)

In [45]:
df_feat_alt_TRN2.first()

Row(nr_idef_secu=u'3246', cd_eqpt=u'151', 1656=0.0, 1657=0.0, 1658=0.0, 1659=0.0, 1660=0.0, 1661=0.0, 1818=0, 1823=0.0, 1884=0.0, 1990=0, 1996=0, 2163=0.0, 2164=0, 2165=0.0, 2169=0, 2215=0.0, 2270=0, 2276=0, 2277=0.0, 2388=0.0, 2389=0.0, 2390=0.0, 2430=0.0, 2433=0.0, 2434=0.0, 2930=0.8313253012048193, 2931=0, 2996=0, 3054=0, 3055=0.0, 3056=0.0, 3107=0.0, 3109=0.0, 3217=0.0, 3219=0, 3320=0.0, 3321=0.0, 3398=0.0, 3399=0.0, 3436=0, 3441=0, 3442=0.0, 3443=0.0, 3551=0.0, 3552=0.0, 3554=0.0, 3559=0.0, 3601=0, 3666=0.0, 3668=0, 3669=0.0, 3712=0, 3718=0, 3771=0, 3821=0.0, 3828=0.0, 3829=0.0, 3893=0.0, 3894=0.0, 3931=0.0, 3936=0, 4050=0, 4109=0.0, 4284=0.0, 4285=0.0, 4328=0, 4329=0.0, 4332=0.0, 4334=0, 4553=0, 4556=0.0, 4557=1.71875, 4558=0.0, 4603=0, 4609=0.0, 5102=0.0, 5103=0.0, 5104=0.0, 5105=0.0, 5106=0, 5109=0.0, 5171=0.0, 5172=0.0, 5174=0, 5223=0, 5225=0, 5281=0.0, 5282=0, 5283=0.0, 5286=0, 5287=0, 5288=0, 5289=0, 5558=0.0, 5559=0.0, 5603=0.0, 5604=0.0, 5605=0.0, 5609=0, 5720=1.0, 5721=0.

In [46]:
# removendo o dataframe Pandas da memoria
lst = [pd_feat_TRN2]
del pd_feat_TRN2 # dfs still in list
del lst     # memory release now

In [47]:
# reordena as colunas
df_feat_alt_TRN2 = df_feat_alt_TRN2.select('nr_idef_secu', 'cd_eqpt','data_even', 
                                           *[col for col in df_feat_alt_TRN2.columns if (col != 'nr_idef_secu') &
                                     (col != 'cd_eqpt') &
                                     (col != 'data_even')])

df_feat_alt_TRN2 = df_feat_alt_TRN2.withColumn('data_even', df_feat_alt_TRN2["data_even"].cast(TimestampType()))

In [48]:
# merge dos dataframes por agencia/eqpt/data
df_feat_alt_TOT = df_feat_alt_TRN1 \
    .join(df_feat_alt_TRN2, (df_feat_alt_TRN1['nr_idef_secu'] == df_feat_alt_TRN2['nr_idef_secu']) &
                      (df_feat_alt_TRN1['cd_eqpt'] == df_feat_alt_TRN2['cd_eqpt']) &
                      (df_feat_alt_TRN1['data_even'] == df_feat_alt_TRN2['data_even']), 'left') \
    .drop(df_feat_alt_TRN2["nr_idef_secu"]).drop(df_feat_alt_TRN2["cd_eqpt"]).drop(df_feat_alt_TRN2["data_even"])

In [ ]:
# FIM PROCESSAMENTO TRN2

In [31]:
# abaixo para testes apenas enquanto nao consegue carregar TRN2
df_feat_alt_TOT = df_feat_alt_TRN1

In [32]:
df_tidy_TOT = df_feat_alt_TOT\
    .withColumn('nr_idef_secu', trim(df_feat_alt_TOT.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zeros  a esquerda

In [33]:
################################# 
df_tidy_TOT = df_tidy_TOT.dropna()

In [34]:
df_poc = df_poc\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zeros  a esquerda

df_poc.printSchema()

root
 |-- data_range: timestamp (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [35]:
df_tidy_TOT.count()

2523

In [36]:
# se achar, marca como POC = '1.0', concatenando por data e agencia
df_tidy_TOT = df_tidy_TOT \
    .join(df_poc, (df_tidy_TOT['nr_idef_secu'] == df_poc['agencia']) &
          (df_tidy_TOT['data_even'] == df_poc['data_range']), 'left')

df_tidy_TOT = df_tidy_TOT.withColumn("label",
        when(df_tidy_TOT["nr_idef_secu"] == df_poc["agencia"],1.0) \
        .otherwise(0.0))\
    .drop(df_poc["agencia"]).drop(df_poc["data_range"]).drop(df_poc["golpe"])

In [37]:
# soma as duas listas obtidas
l_feat_tot = l_features1
#l_feat_tot = l_feat_final + l_feat_final2

In [38]:
# força tipo double nas colunas que a trsnsposição criou como long (motivo desconhecido ainda)
l_primeven = l_feat_tot[0]
df_tidy_dbl_TOT = df_tidy_TOT\
       .withColumn(l_primeven, df_tidy_TOT[l_primeven].cast(DoubleType()))\

for even in l_feat_tot[1:]:
    df_tidy_dbl_TOT = df_tidy_dbl_TOT.withColumn(even, df_tidy_dbl_TOT[even].cast(DoubleType()))

In [39]:
#tabelao_R = '/temporario/tabelao_R.csv' # usada para feature selection no Estatistica (Ronaldo)
#pd = df_tidy_dbl_TRN.toPandas()
#pd.to_csv(tabelao_R)

In [40]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

In [41]:
# Transformar em double as features (features selecionadas)
#df_tidy_dbl = df_tidy\
#    .withColumn('3321', df_tidy["3321"].cast(DoubleType()))\
#    .withColumn('1689', df_tidy["1689"].cast(DoubleType()))\
#    .withColumn('1617', df_tidy["1617"].cast(DoubleType()))\
#    .withColumn('5171', df_tidy["5171"].cast(DoubleType()))\
#    .withColumn('3672', df_tidy["3672"].cast(DoubleType()))\
#    .withColumn('1568', df_tidy["1568"].cast(DoubleType()))\
#    .withColumn('1615', df_tidy["1615"].cast(DoubleType()))\
#   .withColumn('883', df_tidy["883"].cast(DoubleType()))\
#    .withColumn('2143', df_tidy["2143"].cast(DoubleType()))  

In [42]:
# USO
#df_tidy_dbl_USO = df_tidy_USO

## ML 

In [43]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# rodando para as features ocm mais relevância já analisadas
assemblerTrain = VectorAssembler(
    inputCols=l_feat_tot,
    outputCol="features")
#   inputCols=["3321", "1689", "1617","5171", "3672", "1568","1615", "883", "2143"],

df_tmp = assemblerTrain.transform(df_tidy_dbl_TOT)

df_tidy_rf = df_tmp.select("nr_idef_secu", "cd_eqpt", "label", "features")
df_tidy_rf.printSchema()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_rf)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_rf)

# Split the data into training and test sets (30% held out for testing)
#(trainingData_rf, testData_rf) = df_tidy_dt.randomSplit([0.7, 0.3])
(trainData_LPB, testData_LPB) = df_tidy_rf.randomSplit([0.7, 0.3])

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [44]:
#trainData_LPB.groupBy("label").count().orderBy("label").show()

#testData_LPB.groupBy("label").count().orderBy("label").show()

In [45]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# DESCOMENTAR ABAIXO PARA USO DE CROSS VALIDATION
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

numFolds = 2

evaluator = BinaryClassificationEvaluator()

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=numFolds)

model_ml = crossval.fit(trainData_LPB)

# Fetch best model
#tree_model = model.bestModel.stages[2]
#print tree_model

In [46]:
# COMENTAR ABAIXO PARA USO DE CROSS VALIDATION
# Chain indexers and tree in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Train model.  This also runs the indexers.
#model_ml = pipeline.fit(trainData_LPB)

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model_ml.transform(testData_LPB)
predictions.take(2)
#evaluator = BinaryClassificationEvaluator()

[Row(nr_idef_secu=u'1074', cd_eqpt=u'152', label=0.0, features=SparseVector(11, {4: 1.0}), indexedLabel=0.0, indexedFeatures=SparseVector(11, {4: 1.0}), rawPrediction=DenseVector([18.7982, 1.2018]), probability=DenseVector([0.9399, 0.0601]), prediction=0.0),
 Row(nr_idef_secu=u'112', cd_eqpt=u'155', label=1.0, features=SparseVector(11, {3: 0.3913}), indexedLabel=1.0, indexedFeatures=SparseVector(11, {3: 0.3913}), rawPrediction=DenseVector([19.0625, 0.9375]), probability=DenseVector([0.9531, 0.0469]), prediction=0.0)]

In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator.explainParams()

auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print("AUC = %g " % (auroc))

#precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

#print("Precision/Recall = %g " % (precrec))

accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

#Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'

createNewColFromTwo = udf(confusion_matrix, StringType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['label']))

df_rates = df_conf_matr.groupBy("conf_matr").count()
df_rates.toPandas()

predictions.select("nr_idef_secu", "cd_eqpt", "prediction", "indexedLabel", "features", "probability").take(5)

AUC = 0.716607 
Test Error = 0.297248 


[Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', prediction=0.0, indexedLabel=0.0, features=SparseVector(11, {10: 0.8333}), probability=DenseVector([0.9698, 0.0302])),
 Row(nr_idef_secu=u'1206', cd_eqpt=u'156', prediction=0.0, indexedLabel=0.0, features=SparseVector(11, {3: 0.3393}), probability=DenseVector([0.9531, 0.0469])),
 Row(nr_idef_secu=u'21', cd_eqpt=u'154', prediction=0.0, indexedLabel=0.0, features=SparseVector(11, {7: 1.0}), probability=DenseVector([0.9493, 0.0507])),
 Row(nr_idef_secu=u'24', cd_eqpt=u'10001', prediction=0.0, indexedLabel=0.0, features=SparseVector(11, {10: 1.4118}), probability=DenseVector([0.9682, 0.0318])),
 Row(nr_idef_secu=u'344', cd_eqpt=u'158', prediction=0.0, indexedLabel=0.0, features=SparseVector(11, {3: 4.7895}), probability=DenseVector([0.9621, 0.0379]))]

## Uso do modelo

In [51]:
df_eventlog_USO.toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos
0          188   10001         1846 2016-08-20           1
1         4433   10001         1846 2016-08-20           1
2         4664   10001         1846 2016-08-20           1

In [52]:
# USO 2
# concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
df_feat_USO = df_eventlog_USO \
    .join(df_even_med, (df_eventlog_USO['nr_idef_secu'] == df_even_med['nr_idef_secu']) &
                      (df_eventlog_USO['cd_eqpt'] == df_even_med['cd_eqpt']) &
                      (df_eventlog_USO['nr_sequ_even'] == df_even_med['nr_sequ_even']), 'left')\
    .drop(df_even_med["nr_idef_secu"]).drop(df_even_med["cd_eqpt"]).drop(df_even_med["nr_sequ_even"])
#df_feat_USO.take(5)

In [53]:
df_feat_USO.toPandas()

dh_even  nr_eventos nr_idef_secu cd_eqpt nr_sequ_even  med
0 2016-08-20           1          188   10001         1846  1.0

In [54]:
# USO 3
# criar coluna de indice de proximidade da media (iproxmed)
# calculado como número de ocorrências real diaria/média diaria, por agencia e atm
df_feat_USO = df_feat_USO\
   .withColumn('iproxmed', df_feat_USO["nr_eventos"]/df_feat_USO["med"])
#df_feat_USO.toPandas()

In [55]:
df_feat_USO.toPandas()

dh_even  nr_eventos nr_idef_secu cd_eqpt nr_sequ_even  med  iproxmed
0 2016-08-20           1         4433   10001         1846  1.0  1.000000
1 2016-08-20           1         4664   10001         1846  1.0  1.000000
2 2016-08-20           2          188   10001         1846  1.5  1.333333

In [56]:
# USO 4
# transpondo tipo de evento para gerar features
df_feat_USO = df_feat_USO\
    .drop(df_feat_USO["med"]).drop(df_feat_USO["nr_eventos"])
df_feat_USO.toPandas()

dh_even nr_idef_secu cd_eqpt nr_sequ_even  iproxmed
0 2016-08-20         4433   10001         1846  1.000000
1 2016-08-20         4664   10001         1846  1.000000
2 2016-08-20          188   10001         1846  1.555556

In [57]:
df_feat_USO = df_feat_USO.dropna()

pd_tmp_USO = df_feat_USO.select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

#pd_tmp_USO

# USO 5
pd_feat_USO = pd.pivot_table(pd_tmp_USO,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

pd_feat_USO.columns = [c[1] if c[1] else c[0] for c in pd_feat_USO.columns.tolist()]
pd_feat_USO = pd_feat_USO.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat_USO

nr_idef_secu cd_eqpt      1846   data_even
0          188   10001  0.909091  2016-08-20
1         4433   10001  1.000000  2016-08-20
2         4664   10001  1.000000  2016-08-20

In [75]:
df_feat_alt_USO = sqlCtx.createDataFrame(pd_feat_USO)

# USO 6
# reordenando as colunas
df_feat_alt_USO = df_feat_alt_USO.select('nr_idef_secu', 'cd_eqpt','data_even', 
                                           *[col for col in df_feat_alt_USO.columns if (col != 'nr_idef_secu') &
                                     (col != 'cd_eqpt') &
                                     (col != 'data_even')])

df_feat_alt_USO = df_feat_alt_USO.withColumn('data_even', df_feat_alt_USO["data_even"].cast(TimestampType()))
df_feat_alt_USO.toPandas()

nr_idef_secu cd_eqpt  data_even      1846
0          188   10001 2016-08-20  0.909091
1         4433   10001 2016-08-20  1.000000
2         4664   10001 2016-08-20  1.000000

In [82]:
# USO 7
df_tidy_USO = df_feat_alt_USO\
    .withColumn('nr_idef_secu', trim(df_feat_alt_USO.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zeros  a esquerda

In [83]:
df_tidy_dbl_TOT.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- 1649: double (nullable = true)
 |-- 1746: double (nullable = true)
 |-- 1846: double (nullable = true)
 |-- 2184: double (nullable = true)
 |-- 5198: double (nullable = true)
 |-- 5241: double (nullable = true)
 |-- 5242: double (nullable = true)
 |-- 5248: double (nullable = true)
 |-- 5251: double (nullable = true)
 |-- 5840: double (nullable = true)
 |-- 6740: double (nullable = true)
 |-- label: double (nullable = false)



In [84]:
df_tidy_USO.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- 1846: double (nullable = true)



In [86]:
# USO 8
# insere colunas que possam faltar nos dados de uso
for even in list(set(df_tidy_dbl_TOT.columns[3:-1]).difference(df_tidy_USO.columns[3:])):
    df_tidy_USO = df_tidy_USO.withColumn(even, lit(0))
    df_tidy_USO = df_tidy_USO.withColumn(even, df_tidy_USO[even].cast(DoubleType()))

In [87]:
df_tidy_USO.toPandas()

nr_idef_secu cd_eqpt  data_even      1846  5251  5198  1649  2184  5241  \
0          188   10001 2016-08-20  0.909091   0.0   0.0   0.0   0.0   0.0   
1         4433   10001 2016-08-20  1.000000   0.0   0.0   0.0   0.0   0.0   
2         4664   10001 2016-08-20  1.000000   0.0   0.0   0.0   0.0   0.0   

   5242  6740  5840  1746  5248  
0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0

In [88]:
# USO  9
# força label = 0.0 (sem uso, somente para aplicar funcao predict para uso)
df_tidy_USO = df_tidy_USO.withColumn("label", lit(0))
df_tidy_USO = df_tidy_USO.withColumn('label', df_tidy_USO["label"].cast(DoubleType()))
df_tidy_USO.toPandas()

nr_idef_secu cd_eqpt  data_even      1846  5251  5198  1649  2184  5241  \
0          188   10001 2016-08-20  0.909091   0.0   0.0   0.0   0.0   0.0   
1         4433   10001 2016-08-20  1.000000   0.0   0.0   0.0   0.0   0.0   
2         4664   10001 2016-08-20  1.000000   0.0   0.0   0.0   0.0   0.0   

   5242  6740  5840  1746  5248  label  
0   0.0   0.0   0.0   0.0   0.0    0.0  
1   0.0   0.0   0.0   0.0   0.0    0.0  
2   0.0   0.0   0.0   0.0   0.0    0.0

In [89]:
# força tipo double nas colunas que a trsnsposição criou como long (motivo desconhecido ainda)
l_primeven = l_feat_tot[0]
df_tidy_dbl_USO = df_tidy_USO\
       .withColumn(l_primeven, df_tidy_USO[l_primeven].cast(DoubleType()))\

for even in l_feat_tot[1:]:
    df_tidy_dbl_USO = df_tidy_dbl_USO.withColumn(even, df_tidy_dbl_USO[even].cast(DoubleType()))

In [97]:
df_tidy_dbl_USO.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- 1846: double (nullable = true)
 |-- 5251: double (nullable = false)
 |-- 5198: double (nullable = false)
 |-- 1649: double (nullable = false)
 |-- 2184: double (nullable = false)
 |-- 5241: double (nullable = false)
 |-- 5242: double (nullable = false)
 |-- 6740: double (nullable = false)
 |-- 5840: double (nullable = false)
 |-- 1746: double (nullable = false)
 |-- 5248: double (nullable = false)
 |-- label: double (nullable = false)



In [98]:
# colocar aqui o uso conforme no ML
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#predictions = model_ml.transform(testData_LPB)
#predictions.take(2)
#evaluator = BinaryClassificationEvaluator()

#df_tidy_mllib = df_tidy_dbl_USO.select('nr_idef_secu', 'cd_eqpt','data_even', 'label', 
#                                           *[col for col in df_tidy_dbl_USO.columns if (col != 'label') &
#                                     (col != 'nr_idef_secu') &
##                                     (col != 'cd_eqpt') &
#                                     (col != 'data_even') &
#                                     (col != 'golpe')])

#df_tidy_mllib.limit(10).toPandas()

In [96]:
df_tidy_dbl_TOT.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- 1649: double (nullable = true)
 |-- 1746: double (nullable = true)
 |-- 1846: double (nullable = true)
 |-- 2184: double (nullable = true)
 |-- 5198: double (nullable = true)
 |-- 5241: double (nullable = true)
 |-- 5242: double (nullable = true)
 |-- 5248: double (nullable = true)
 |-- 5251: double (nullable = true)
 |-- 5840: double (nullable = true)
 |-- 6740: double (nullable = true)
 |-- label: double (nullable = false)



In [109]:
df_tmp = assemblerTrain.transform(df_tidy_dbl_USO)

df_tidy_uso = df_tmp.select("nr_idef_secu", "cd_eqpt", "data_even", "label", "features")
df_tidy_uso.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [110]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_uso)

In [111]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_uso)

# Split the data into training and test sets (30% held out for testing)
#(trainingData_rf, testData_rf) = df_tidy_dt.randomSplit([0.7, 0.3])
#(trainData_LPB, testData_LPB) = df_tidy_rf.randomSplit([0.7, 0.3])

In [112]:
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model_ml.transform(df_tidy_uso)
predictions.take(2)

[Row(nr_idef_secu=u'4433', cd_eqpt=u'10001', data_even=datetime.datetime(2016, 8, 20, 0, 0), label=0.0, features=SparseVector(11, {10: 1.0}), indexedLabel=0.0, indexedFeatures=SparseVector(11, {10: 1.0}), rawPrediction=DenseVector([19.3376, 0.6624]), probability=DenseVector([0.9669, 0.0331]), prediction=0.0),
 Row(nr_idef_secu=u'4664', cd_eqpt=u'10001', data_even=datetime.datetime(2016, 8, 20, 0, 0), label=0.0, features=SparseVector(11, {10: 1.0}), indexedLabel=0.0, indexedFeatures=SparseVector(11, {10: 1.0}), rawPrediction=DenseVector([19.3376, 0.6624]), probability=DenseVector([0.9669, 0.0331]), prediction=0.0)]

In [127]:
predictions.map(lambda p: (p[2],p[0],p[1],p[3], p[8][1] )).first()

(datetime.datetime(2016, 8, 20, 0, 0),
 u'188',
 u'10001',
 0.0,
 0.033122405852504852)

In [146]:
df_rankingFinal = predictions.map(lambda p: (p[2],p[0],p[1],p[3], str(p[8][1]) ))\
    .toDF(['dh_even',
           'nr_idef_secu', 
           'cd_eqpt', 
           'label',
           'probabilidade'
                   ])

In [155]:
from pyspark.sql.functions import col,udf, unix_timestamp
df_rankingFinal = df_rankingFinal\
    .withColumn('probabilidade', df_rankingFinal["probabilidade"].cast(DoubleType()))\
    .orderBy(col('probabilidade').desc())

In [157]:
df_rankingFinal.toPandas()

dh_even nr_idef_secu cd_eqpt  label  probabilidade
0 2016-08-20          188   10001    0.0       0.033122
1 2016-08-20         4664   10001    0.0       0.033122
2 2016-08-20         4433   10001    0.0       0.033122

In [165]:
# salvando ranking
df_rankingFinal.toPandas().to_csv(ranking_poc_atm)